In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
import json
import pickle
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json', encoding = 'utf8').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [4]:
words = [word for word in words if word.isalpha()]
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
words = [lemmatizer.lemmatize(w.lower(), pos = 'v') for w in words if w not in ignore_words]
words = [word for word in words if len(word)>1]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

In [5]:
training = []
output_empty = [0]*len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower(), pos = 'v') for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype = object)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 600, batch_size = 5, verbose = 1)
model.save('model.h5', hist)

Epoch 1/600
99/99 [==============================] - 0s 2ms/step - loss: 3.5918 - accuracy: 0.0528
Epoch 2/600
99/99 [==============================] - 0s 1ms/step - loss: 3.4167 - accuracy: 0.0955 ETA: 0s - loss: 3.4276 - accuracy: 0.09
Epoch 3/600
99/99 [==============================] - 0s 1ms/step - loss: 3.2726 - accuracy: 0.1199
Epoch 4/600
99/99 [==============================] - 0s 1ms/step - loss: 3.0400 - accuracy: 0.1748
Epoch 5/600
99/99 [==============================] - 0s 1ms/step - loss: 2.8297 - accuracy: 0.2134
Epoch 6/600
99/99 [==============================] - 0s 1ms/step - loss: 2.5555 - accuracy: 0.2967
Epoch 7/600
99/99 [==============================] - 0s 1ms/step - loss: 2.3411 - accuracy: 0.3598
Epoch 8/600
99/99 [==============================] - 0s 2ms/step - loss: 2.2127 - accuracy: 0.3862
Epoch 9/600
99/99 [==============================] - 0s 2ms/step - loss: 1.9256 - accuracy: 0.4553
Epoch 10/600
99/99 [==============================] - 0s 1ms/step - l

99/99 [==============================] - 0s 2ms/step - loss: 0.6864 - accuracy: 0.8211
Epoch 83/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5950 - accuracy: 0.8516
Epoch 84/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5110 - accuracy: 0.8598
Epoch 85/600
99/99 [==============================] - 0s 1ms/step - loss: 0.4606 - accuracy: 0.8760
Epoch 86/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5537 - accuracy: 0.8699
Epoch 87/600
99/99 [==============================] - 0s 1ms/step - loss: 0.4809 - accuracy: 0.8679
Epoch 88/600
99/99 [==============================] - 0s 1ms/step - loss: 0.4724 - accuracy: 0.8720
Epoch 89/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5768 - accuracy: 0.8476
Epoch 90/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6177 - accuracy: 0.8577
Epoch 91/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6139 - accuracy: 0.8476
Epoch 92/600


99/99 [==============================] - 0s 1ms/step - loss: 1.0768 - accuracy: 0.8130
Epoch 164/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6508 - accuracy: 0.8354
Epoch 165/600
99/99 [==============================] - 0s 1ms/step - loss: 0.8352 - accuracy: 0.8191
Epoch 166/600
99/99 [==============================] - 0s 1ms/step - loss: 1.0909 - accuracy: 0.7927
Epoch 167/600
99/99 [==============================] - 0s 2ms/step - loss: 0.8962 - accuracy: 0.8028
Epoch 168/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7562 - accuracy: 0.8252
Epoch 169/600
99/99 [==============================] - 0s 1ms/step - loss: 0.9237 - accuracy: 0.8049
Epoch 170/600
99/99 [==============================] - 0s 1ms/step - loss: 0.9129 - accuracy: 0.8049
Epoch 171/600
99/99 [==============================] - 0s 1ms/step - loss: 0.9766 - accuracy: 0.7927
Epoch 172/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6738 - accuracy: 0.8354
Epoc

99/99 [==============================] - 0s 1ms/step - loss: 0.8419 - accuracy: 0.8008
Epoch 245/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7412 - accuracy: 0.8089
Epoch 246/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7637 - accuracy: 0.7988
Epoch 247/600
99/99 [==============================] - 0s 1ms/step - loss: 0.8229 - accuracy: 0.7907
Epoch 248/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7882 - accuracy: 0.8272
Epoch 249/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7575 - accuracy: 0.8150
Epoch 250/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6391 - accuracy: 0.8211
Epoch 251/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7251 - accuracy: 0.8232
Epoch 252/600
99/99 [==============================] - 0s 1ms/step - loss: 0.8497 - accuracy: 0.8150
Epoch 253/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7775 - accuracy: 0.8252
Epoc

99/99 [==============================] - 0s 1ms/step - loss: 0.6918 - accuracy: 0.8191
Epoch 325/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7097 - accuracy: 0.8110
Epoch 326/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6807 - accuracy: 0.8150
Epoch 327/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7753 - accuracy: 0.8110
Epoch 328/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5418 - accuracy: 0.8557
Epoch 329/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7341 - accuracy: 0.8537
Epoch 330/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7349 - accuracy: 0.8211
Epoch 331/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5548 - accuracy: 0.8496
Epoch 332/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7675 - accuracy: 0.8130
Epoch 333/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6885 - accuracy: 0.8272
Epoc

99/99 [==============================] - 0s 1ms/step - loss: 0.5848 - accuracy: 0.8354
Epoch 406/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7591 - accuracy: 0.8150
Epoch 407/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7074 - accuracy: 0.8272
Epoch 408/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6693 - accuracy: 0.8171
Epoch 409/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5846 - accuracy: 0.8354
Epoch 410/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6467 - accuracy: 0.8191
Epoch 411/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7840 - accuracy: 0.8089
Epoch 412/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6986 - accuracy: 0.8374
Epoch 413/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6764 - accuracy: 0.8150
Epoch 414/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5665 - accuracy: 0.8496
Epoc

99/99 [==============================] - 0s 1ms/step - loss: 0.8200 - accuracy: 0.8232
Epoch 487/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6941 - accuracy: 0.8171
Epoch 488/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5869 - accuracy: 0.8354
Epoch 489/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7347 - accuracy: 0.8069
Epoch 490/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7368 - accuracy: 0.8008
Epoch 491/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6699 - accuracy: 0.8293
Epoch 492/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7134 - accuracy: 0.8252
Epoch 493/600
99/99 [==============================] - 0s 1ms/step - loss: 0.8081 - accuracy: 0.8008
Epoch 494/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6644 - accuracy: 0.8293
Epoch 495/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6315 - accuracy: 0.8313
Epoc

99/99 [==============================] - 0s 1ms/step - loss: 0.6635 - accuracy: 0.8333
Epoch 568/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6263 - accuracy: 0.8557
Epoch 569/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7499 - accuracy: 0.8089
Epoch 570/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6952 - accuracy: 0.8211
Epoch 571/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5933 - accuracy: 0.8415
Epoch 572/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6875 - accuracy: 0.8394
Epoch 573/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6884 - accuracy: 0.8333
Epoch 574/600
99/99 [==============================] - 0s 1ms/step - loss: 0.6487 - accuracy: 0.8272
Epoch 575/600
99/99 [==============================] - 0s 1ms/step - loss: 0.7308 - accuracy: 0.8232
Epoch 576/600
99/99 [==============================] - 0s 1ms/step - loss: 0.5583 - accuracy: 0.8598
Epoc